<a href="https://colab.research.google.com/github/Amonwill/RedNeuronalAloeVera/blob/main/AloeveraRN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Descarga del dataset
import kagglehub
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
import os

path = kagglehub.dataset_download("rubab123/aloe-vera-images")
print("Dataset en:", path)

# 2. Parámetros
IMG_SIZE = 150
BATCH_SIZE = 32

# 3. Generadores
datagen_train = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.25,
    height_shift_range=0.25,
    zoom_range=[0.5, 1.5],
    validation_split=0.2
)

datagen_val = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_gen = datagen_train.flow_from_directory(
    path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_gen = datagen_val.flow_from_directory(
    path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

# 4. Modelo optimizado para móvil
modelo = tf.keras.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.GlobalAveragePooling2D(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])

# 5. Compilación
modelo.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# 6. Entrenamiento con EarlyStopping
early = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

history = modelo.fit(
    train_gen,
    epochs=30,
    validation_data=val_gen,
    callbacks=[early]
)

# 7. Guardar modelo Keras
modelo.save("aloe_modelo.h5")

# 8. Conversión a TFLite optimizado
converter = tf.lite.TFLiteConverter.from_keras_model(modelo)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open("modelo_aloe.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ Modelo TFLite generado correctamente")

# 9. Verificación
print("Existe el archivo:", os.path.exists("modelo_aloe.tflite"))
print("Tamaño:", os.path.getsize("modelo_aloe.tflite") / 1024, "KB")

# 10. Descarga automática (SOLO Google Colab)
try:
    from google.colab import files
    files.download("modelo_aloe.tflite")
except:
    print("No estás en Google Colab, el archivo quedó guardado localmente.")


Using Colab cache for faster access to the 'aloe-vera-images' dataset.
Dataset en: /kaggle/input/aloe-vera-images
Found 2796 images belonging to 1 classes.
Found 699 images belonging to 1 classes.
Epoch 1/30
88/88 ━━━━━━━━━━━━━━━━━━━━ 134s 1s/step - accuracy: 1.0000 - loss: 0.1102 - val_accuracy: 1.0000 - val_loss: 3.1554e-12
Epoch 2/30
88/88 ━━━━━━━━━━━━━━━━━━━━ 131s 1s/step - accuracy: 1.0000 - loss: 3.7983e-08 - val_accuracy: 1.0000 - val_loss: 2.8165e-12
Epoch 3/30
88/88 ━━━━━━━━━━━━━━━━━━━━ 130s 1s/step - accuracy: 1.0000 - loss: 9.1968e-08 - val_accuracy: 1.0000 - val_loss: 2.2923e-12
Epoch 4/30
88/88 ━━━━━━━━━━━━━━━━━━━━ 129s 1s/step - accuracy: 1.0000 - loss: 3.5210e-08 - val_accuracy: 1.0000 - val_loss: 2.0821e-12
Epoch 5/30
88/88 ━━━━━━━━━━━━━━━━━━━━ 130s 1s/step - accuracy: 1.0000 - loss: 2.4727e-08 - val_accuracy: 1.0000 - val_loss: 1.9541e-12
Epoch 6/30
88/88 ━━━━━━━━━━━━━━━━━━━━ 130s 1s/step - accuracy: 1.0000 - loss: 1.5987e-08 - val_accuracy: 1.0000 - val_loss: 1.8857e-

Saved artifact at '/tmp/tmpcunur_n_'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name='keras_tensor_18')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  138211122617936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138211122620432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138211122621392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138211122620624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138211122621776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138211122621584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138211122622160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138211122621968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138211122622544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138211122622352: TensorSpec(shape=(), dtype=tf.resource, name=None)
✅ Modelo TFLi

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>